In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from difflib import get_close_matches

from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

In [2]:
!jt -t grade3

In [3]:
arb18 = pd.read_csv('arb18.csv')
arb19 = pd.read_csv('arb19.csv')
arb20 = pd.read_csv('arb20.csv')
arb21 = pd.read_csv('arb21.csv')
arb22 = pd.read_csv('arb22.csv')
arb23 = pd.read_csv('arb23.csv')

# Clean Data

In [4]:
def clean_data(data):
    filter_cols = ['Player', 'Season', 'Club', "Pos'n", 
                   'MLS','Prev. Salary', 'New Salary']
    data = data[filter_cols].iloc[1:,:]
    
    
    # Mapping league minimum values based on season
    salary_mapping = {
        2018: 545000, 2019: 555000, 2020: 563500, 2021: 570500, 2022: 700000, 2023: 720000
    }
    
    data['Prev. Salary'] = pd.to_numeric(data['Prev. Salary'].str.replace(
        '[\$,]', '', regex=True), errors='coerce')
    data['New Salary'] = pd.to_numeric(data['New Salary'].str.replace(
        '[\$,]', '', regex=True), errors='coerce')
    
    # Fill NA values of previous salary with league minimum for seasons
    data['Prev. Salary'] = data.apply(
        lambda row: salary_mapping[row['Season']] if pd.isna(row['Prev. Salary']) else row['Prev. Salary'], axis=1)

    ret_data = data.drop('Club',axis=1)
    
    ret_data['Player'] = ret_data.apply(lambda row: row['Player'].split(', ')[1] + ' ' + row['Player'].split(', ')[0], axis = 1)
    ret_data['Season'] = ret_data['Season'] -1
    ret_data['MLS'] = ret_data['MLS'].round(0)
    return ret_data.iloc[1:,:]
    
    

In [5]:
arb18_clean = clean_data(arb18)
arb19_clean = clean_data(arb19)
arb20_clean = clean_data(arb20)
arb21_clean = clean_data(arb21)
arb22_clean = clean_data(arb22)
arb23_clean = clean_data(arb23)

In [6]:
all_arb_data = pd.concat([arb18_clean,arb19_clean,arb20_clean,
                          arb21_clean,arb22_clean,arb23_clean]).dropna()

# WAR, career WAR Data
https://www.fangraphs.com/leaders/major-league?pos=all&stats=pit&lg=all&type=c%2C7%2C8%2C13%2C59&ind=1&v_cr=202301&startdate=&enddate=&month=0&qual=1&season1=2010&season=2023

In [7]:
sp_arb = all_arb_data[all_arb_data["Pos'n"].isin(['rhp-s','lhp-s'])]

In [8]:
war_data = pd.read_csv('war_data.csv')
# Custom function to replace decimals
def replace_decimals(value):
    decimal_part = round(value - int(value),1)
    if decimal_part == 0.1:
        return int(value) + 0.33
    elif decimal_part == 0.2:
        return int(value) + 0.66
    else:
        return value

# Apply the custom function to the specified column
war_data['IP'] = war_data.apply(lambda row: replace_decimals(row['IP']), axis=1)

war_data = war_data.rename(columns = {'Name': 'Player'})
war_df = war_data[['Season','Player','GS','IP','WAR']].sort_values(['Player','Season'])

In [9]:
find_closest_match = lambda name, choices: get_close_matches(name, choices, n=1)[0]
# Apply the function to find the closest match
sp_arb['Player'] = sp_arb['Player'].apply(lambda x: find_closest_match(x, list(war_df['Player'])))

# Calculate the cumulative sum for each pitcher
war_df['Cumu_WAR'] = war_df.sort_values(['Player','Season']).groupby('Player')['WAR'].cumsum()
# Merge the two DataFrames based on 'Player' and 'Seaons'
merged_df = pd.merge(sp_arb, war_df, on=['Player', 'Season'], how='left').dropna()

# Linear Regression of Salaries, per Year

In [10]:
merged_df.MLS.value_counts()

3.0    93
4.0    74
5.0    49
2.0    24
Name: MLS, dtype: int64

In [11]:
X_cols = ['Prev. Salary','GS','IP','WAR','Cumu_WAR']
y_col = ['New Salary']

fourth_year = merged_df[merged_df.MLS == 3]
fifth_year = merged_df[merged_df.MLS == 4]
sixth_year = merged_df[merged_df.MLS == 5]

In [12]:
lin_reg_arb1 = LinearRegression().fit(fourth_year[X_cols], fourth_year[y_col])
lin_reg_arb2 = LinearRegression().fit(fifth_year[X_cols], fifth_year[y_col])
lin_reg_arb3 = LinearRegression().fit(sixth_year[X_cols], sixth_year[y_col])

# Predictions on Jesús Luzardo

Assumptions:
   - Luzardo will produce ~10 WAR over next three seasons (3.3 projected fWAR in 2024), average 3.3 WAR
   - Luzardo will throw 180 IP each of next three seasons with 30 GS

In [13]:
def round_salary(number):
    return round(number / 100000) * 100000

In [14]:
luzardo_23 = war_df[(war_df.Player == 'Jesús Luzardo') & (war_df.Season == 2023)].reset_index(drop=True)
luzardo_23.loc[0,'Prev. Salary'] = 2650000
luzardo_23 = luzardo_23[['Prev. Salary','GS','IP','WAR','Cumu_WAR']]

In [15]:
fourth_year_pred = round_salary(lin_reg_arb1.predict(luzardo_23)[0][0])
print('Jesús Luzardo, 4th year salary:', fourth_year_pred)

fourth_year_sv = round_salary(int((luzardo_23['WAR']*8000000) - fourth_year_pred))
print('Jesús Luzardo, 4th year Surplus Value:', fourth_year_sv)

Jesús Luzardo, 4th year salary: 5400000
Jesús Luzardo, 4th year Surplus Value: 24300000


In [16]:
fifth_year_input = [[fourth_year_pred, 30, 180, 3.3, luzardo_23['Cumu_WAR'][0] + 3.3]]
fifth_year_pred = round_salary(lin_reg_arb2.predict(fifth_year_input)[0][0])
print('Jesús Luzardo, 5th year salary:', fifth_year_pred)

fifth_year_sv = round_salary(int((3.3*8000000) - fifth_year_pred))
print('Jesús Luzardo, 5th year Surplus Value:', fifth_year_sv)

Jesús Luzardo, 5th year salary: 8700000
Jesús Luzardo, 5th year Surplus Value: 17700000


In [17]:
sixth_year_input = [[fifth_year_pred, 30, 180, 3.3, luzardo_23['Cumu_WAR'][0] + 6.6]]
sixth_year_pred = round_salary(lin_reg_arb3.predict(sixth_year_input)[0][0])
print('Jesús Luzardo, 6th year salary:', sixth_year_pred)

sixth_year_sv = round_salary(int((3.3*8000000) - sixth_year_pred))
print('Jesús Luzardo, 5th year Surplus Value:', sixth_year_sv)

Jesús Luzardo, 6th year salary: 14400000
Jesús Luzardo, 5th year Surplus Value: 12000000


In [18]:
print('Total Surplus Value:', fourth_year_sv + fifth_year_sv + sixth_year_sv)

Total Surplus Value: 54000000
